# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import time
from ratelimit import limits, sleep_and_retry

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [18]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
lat = []
lng = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500) 
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500) 
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
cities_df = pd.DataFrame({'cities':cities})
# Print the city count to confirm sufficient count
print(len(cities))
cities_df.head()

623


,cities
0,sur
1,ixtapa
2,rikitea
3,gerash
4,hobart


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [19]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial" #Fahrenheit

# Build partial query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

# set up lists to hold reponse info
cities_list = []
lat = []
lng = []
temp_max = []
humidity = []
cloudiness = []
wind_speed = []
country=[]
date = []

In [20]:
record=0

for city in cities:
    try:
        response = requests.get(query_url + city).json()
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        temp_max.append(response['main']['temp_max'])
        humidity.append(response['main']['humidity'])
        cloudiness.append(response['clouds']['all'])
        wind_speed.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        cities_list.append(city)
        record += 1
        print(f"Processing Record {record} for city = {city}")
        time.sleep(1)
    except KeyError:
        print("City not found...skipping record")

Processing Record 1 for city = sur
Processing Record 2 for city = ixtapa
Processing Record 3 for city = rikitea
Processing Record 4 for city = gerash
Processing Record 5 for city = hobart
Processing Record 6 for city = ilulissat
Processing Record 7 for city = bamora
Processing Record 8 for city = constitucion
Processing Record 9 for city = arraial do cabo
Processing Record 10 for city = wainwright
Processing Record 11 for city = ushuaia
Processing Record 12 for city = hovd
Processing Record 13 for city = taltal
Processing Record 14 for city = atuona
Processing Record 15 for city = hithadhoo
City not found...skipping record
City not found...skipping record
City not found...skipping record
Processing Record 16 for city = albany
Processing Record 17 for city = zhigansk
Processing Record 18 for city = bud
Processing Record 19 for city = invermere
Processing Record 20 for city = bluff
Processing Record 21 for city = labuhan
Processing Record 22 for city = valdivia
Processing Record 23 for c

Processing Record 181 for city = yushu
Processing Record 182 for city = cidreira
Processing Record 183 for city = tete
Processing Record 184 for city = atar
Processing Record 185 for city = parangan
Processing Record 186 for city = warwick
Processing Record 187 for city = port elizabeth
Processing Record 188 for city = greeley
Processing Record 189 for city = morondava
Processing Record 190 for city = chumikan
Processing Record 191 for city = prince george
Processing Record 192 for city = mangrol
Processing Record 193 for city = jamestown
Processing Record 194 for city = pendleton
Processing Record 195 for city = shache
Processing Record 196 for city = dothan
Processing Record 197 for city = tocopilla
Processing Record 198 for city = bereznehuvate
Processing Record 199 for city = seoul
Processing Record 200 for city = rapid valley
Processing Record 201 for city = yellowknife
Processing Record 202 for city = ostrovnoy
Processing Record 203 for city = palmas
Processing Record 204 for cit

Processing Record 369 for city = alappuzha
City not found...skipping record
Processing Record 370 for city = atikokan
Processing Record 371 for city = buea
Processing Record 372 for city = vrangel
Processing Record 373 for city = kieta
City not found...skipping record
Processing Record 374 for city = champerico
Processing Record 375 for city = mana
Processing Record 376 for city = linxia
Processing Record 377 for city = karpinsk
Processing Record 378 for city = eureka
Processing Record 379 for city = isangel
City not found...skipping record
City not found...skipping record
Processing Record 380 for city = labytnangi
City not found...skipping record
Processing Record 381 for city = havoysund
Processing Record 382 for city = marzuq
City not found...skipping record
Processing Record 383 for city = beloha
Processing Record 384 for city = luganville
Processing Record 385 for city = kamina
Processing Record 386 for city = san cristobal
Processing Record 387 for city = rangia
Processing Recor

Processing Record 548 for city = awjilah
Processing Record 549 for city = achikulak
Processing Record 550 for city = biloela
Processing Record 551 for city = tucurui
Processing Record 552 for city = flinders
Processing Record 553 for city = kuleshovka
Processing Record 554 for city = terra santa
Processing Record 555 for city = kalbugan
Processing Record 556 for city = blackwater
Processing Record 557 for city = koungou
Processing Record 558 for city = tiznit
Processing Record 559 for city = conceicao do mato dentro
City not found...skipping record
Processing Record 560 for city = praya
Processing Record 561 for city = debe
Processing Record 562 for city = sept-iles
Processing Record 563 for city = olafsvik
Processing Record 564 for city = ketchikan
Processing Record 565 for city = nassau
Processing Record 566 for city = puerto del rosario
Processing Record 567 for city = liverpool
Processing Record 568 for city = salvador
Processing Record 569 for city = mayo
Processing Record 570 for

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [24]:
weather_dict = {
    "City": cities_list,
    "Lat": lat,
    "Lng": lng,
    "Max Temp": temp_max,
    "Humidity": humidity,
    "cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country": country,
    "Date": date
}

weather_data_df = pd.DataFrame(weather_dict)

weather_data_df.to_csv(r'weather_data.csv',index=False)
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,cloudiness,Wind Speed,Country,Date
0,sur,22.57,59.53,74.95,59,0,7.70,OM,1607311766
1,ixtapa,20.70,-105.20,72.00,88,90,3.47,MX,1607312366
2,rikitea,-23.12,-134.97,76.96,80,0,15.35,PF,1607311936
3,gerash,27.67,54.14,57.20,87,40,6.93,IR,1607312368
4,hobart,-42.88,147.33,60.80,38,75,19.46,AU,1607311882


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [31]:
weather_data_humidity_df = weather_data_df.loc[weather_data_df["Humidity"] > 100]
len(weather_data_humidity_df)

0

In [28]:
#  Get the indices of cities that have humidity over 100%.
print("no cities with >100% humidity")

no cities with >100% humidity


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression